<a href="https://colab.research.google.com/github/AMEYABHAVE/Cristl/blob/main/project2_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.6.0


In [2]:
from google.colab import drive
import pandas as pd
file_location = '/content/drive/MyDrive/accelerometer.csv'
accelerometer =pd.read_csv(file_location)
accelerometer.drop(columns=['time','repetition','uuid'], inplace=True)
accchair= accelerometer[accelerometer['activity'] == 'CHAIR'].head(31065)
acccouch= accelerometer[accelerometer['activity'] == 'COUCH'].head(31065)
accsurface= accelerometer[accelerometer['activity'] == 'SURFACE'].head(31065)
acclyingup= accelerometer[accelerometer['activity'] == 'LYING_UP'].head(31065)
acclyingside= accelerometer[accelerometer['activity'] == 'LYING_SIDE'].head(31065)
accdataset = pd.DataFrame()
accdataset = accdataset.append([accchair, acccouch, accsurface, acclyingup, acclyingside])

file_location1 = '/content/drive/MyDrive/gyroscope.csv'
gyroscope =pd.read_csv(file_location1)
gyroscope.drop(columns=['time','repetition','uuid'], inplace=True)
gyrchair= gyroscope[gyroscope['activity'] == 'CHAIR'].head(10178)
gyrcouch= gyroscope[gyroscope['activity'] == 'COUCH'].head(10178)
gyrsurface= gyroscope[gyroscope['activity'] == 'SURFACE'].head(10178)
gyrlyingup= gyroscope[gyroscope['activity'] == 'LYING_UP'].head(10178)
gyrlyingside= gyroscope[gyroscope['activity'] == 'LYING_SIDE'].head(10178)
gyrdataset = pd.DataFrame()
gyrdataset = gyrdataset.append([gyrchair, gyrcouch, gyrsurface, gyrlyingup, gyrlyingside])

In [3]:
accdataset.info()
gyrdataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155325 entries, 0 to 290908
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   name      155325 non-null  object 
 1   acc_x     155325 non-null  float64
 2   acc_y     155325 non-null  float64
 3   acc_z     155325 non-null  float64
 4   activity  155325 non-null  object 
dtypes: float64(3), object(2)
memory usage: 7.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50890 entries, 0 to 93659
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      50890 non-null  object 
 1   gyr_x     50890 non-null  float64
 2   gyr_y     50890 non-null  float64
 3   gyr_z     50890 non-null  float64
 4   activity  50890 non-null  object 
dtypes: float64(3), object(2)
memory usage: 2.3+ MB
